In [ ]:
import os
import numpy as np
import nibabel as nib
import SimpleITK as sitk

def registration(fix_path, moving_path, output_path):
    
    # read files
    fixed_image = sitk.ReadImage(fix_path, sitk.sitkFloat64)
    moving_image = sitk.ReadImage(moving_path, sitk.sitkFloat64)
    
    # define parameters for registration
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)  
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkBSpline3)
    registration_method.SetOptimizerAsGradientDescent(
        learningRate=1.0,
        numberOfIterations=100,
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()    
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    final_transform = registration_method.Execute(fixed_image , moving_image)
    print(registration_method.GetOptimizerStopConditionDescription())
    
    # resample and save the source image
    out = sitk.Resample(moving_image, fixed_image, final_transform, 
                        sitk.sitkBSpline, 0.0, moving_image.GetPixelID())    
    out = sitk.Cast(out, sitk.sitkFloat64)    
    sitk.WriteImage(out, output_path)

In [ ]:
template_path = 'path and filename of nifti file that would like to be template'
source_path = 'path and filename of nifti file that you want to registrate to template'
output_path = 'path and filename of affined nifti file'

registration(template_path, source_path, output_path)